This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [79]:
# First, import the relevant modules
import requests
from datetime import datetime

In [19]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = f"https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key={API_KEY}"
r = requests.get(f"{url}&end_date=2021-01-11&limit=10")

In [20]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data = r.json()
print(data)

{'dataset_data': {'limit': 10, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2020-12-01', 'frequency': 'daily', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None], ['2020-11-30', 111.0, 113.6, 111.0, 112.1, None, 315.0, 35111.5, None, None, None], ['2020-11-27', 111.0, 111.5, 109.5, 111.2, None, 155.0, 17168.5, None, None, None], ['2020-11-26', 111.3, 112.0, 110.5, 111.6, None, 254.0, 28367.5, None, None, None], ['2020-11-25', 111.4, 111.4, 110.1, 110.1, None, 294.0, 32435.8, None, None, None], ['2020-11-24', 116.1, 116.8, 110.5, 113.5, None, 3184.0, 361404.4, None, None, None], ['2020-11-23', 118.0, 118.8, 116.3, 116.3, None, 1590.0, 187525.0, None, None, None], ['2020-11-20', 116.5, 118.0, 116.5, 117.5, None, 295.0, 34736.0, None, None, None], ['2020-

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [28]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
#2. Convert the returned JSON object into a Python dictionary.
r = requests.get(f"https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key={API_KEY}&start_date=2017-01-01&end_date=2017-12-31&frequency=daily")
stocks = r.json()

In [80]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.
date_idx = stocks['dataset']['column_names'].index('Date')
open_idx = stocks['dataset']['column_names'].index('Open')
data = sorted(stocks['dataset']['data'], key=lambda day: datetime.strptime(day[date_idx], '%Y-%m-%d'))
open_prices = [day[open_idx] for day in data if day[open_idx] != None]
print(max(open_prices), min(open_prices))

53.11 34.0


In [95]:
#4. What was the largest change in any one day (based on High and Low price)?
high_idx = stocks['dataset']['column_names'].index('High')
low_idx =  stocks['dataset']['column_names'].index('Low')
total_prices = len(data)
changes = []
for i in range(0, total_prices):
    if i == total_prices-1:
        break
    if data[i][high_idx] == None or data[i+1][low_idx] == None:
        continue
        
    high_price = data[i][high_idx]
    day_diffs = []
    for j in range(i+1, total_prices):        
        low_price = data[j][low_idx]
        day_diffs.append(abs(high_price - low_price))
    
    changes.append(max(day_diffs))

print(max(changes))

18.36


In [93]:
#5. What was the largest change between any two days (based on Closing Price)?
close_idx = stocks['dataset']['column_names'].index('Close')
closing_changes = []
for i in range(0, total_prices):
    if i == total_prices-2:
        break
    if data[i][close_idx] == None or data[i+2][close_idx] == None:
        continue
        
    close_price1 = data[i][close_idx]
    day_diffs = []
    for j in range(i+2, total_prices):        
        close_price2 = data[j][close_idx]
        day_diffs.append(abs(close_price2 - close_price1))
    
    closing_changes.append(max(day_diffs))
    
#2.559999999999995    
print(max(closing_changes))

19.03


In [96]:
#6. What was the average daily trading volume during this year?
volume_idx = stocks['dataset']['column_names'].index('Traded Volume')
volumes = [day[volume_idx] for day in data if day[volume_idx] != None]
avg_traded_vol = sum(volumes)/len(volumes)
print(avg_traded_vol)

89124.33725490196


In [100]:
#7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
volume_idx = stocks['dataset']['column_names'].index('Traded Volume')
volumes = sorted([day[volume_idx] for day in data if day[volume_idx] != None])
total_volumes = len(volumes)
mid_idx = total_volumes//2
median_traded_vol = volumes[mid_idx] if total_volumes % 2 != 0 else (volumes[mid_idx] + volumes[mid_idx+1])/2
print(median_traded_vol)

76286.0
